# Experiment PAMAP2 with mcfly

This turorial is intended to talk you through the functionalities of mcfly. As an example dataset we use the publicly available PAMAP2 dataset. It contains time series data from movement sensors worn by nine individuals. The data is labelled with the activity types that these individuals did and the aim is to train and evaluate a classifier.

Before you can start, please make sure you installed all the dependencies of mcfly (listed in requirements.txt) and make sure your jupyter notebook has a python3 kernel.

## Import required Python modules

In [2]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
import numpy as np
import pandas as pd
# mcfly
from mcfly import tutorial_pamap2, modelgen, find_architecture, storage
# Keras module is use for the deep learning
import keras
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution1D, Flatten, MaxPooling1D
from keras.optimizers import Adam
# We can set some backend options to avoid NaNs
from keras import backend as K

Using Theano backend.


## Load the data

In [3]:
datapath = '/media/sf_VBox_Shared/timeseries/PAMAP2_Dataset/slidingwindow512cleaned/'
Xs = []
ys = []

ext = '.npy'
for i in range(9):
    Xs.append(np.load(datapath+'X_train_'+str(i)+ext))
    ys.append(np.load(datapath+'y_train_binary'+str(i)+ext))

## Generate models

First step is to create a model architecture. As we do not know what architecture is best for our data we will create a set of models to investigate which architecture is most suitable for our data and classification task. You will need to specificy how many models you want to create with argument 'number_of_models', the type of model which can been 'CNN' or 'DeepConvLSTM', and maximum number of layers per modeltype. See for a full overview of the optional arguments the function documentation of modelgen.generate_models

In [5]:
num_classes = ys[0].shape[1]
np.random.seed(123)
models = modelgen.generate_models(Xs[0].shape,
                                  number_of_classes=num_classes,
                                  number_of_models = 25)

## Compare models
Now that the model architectures have been generated it is time to compare the models by training them in a subset of the training data and evaluating the models in the validation subset. This will help us to choose the best candidate model. Performance results are stored in a json file.

In [7]:
# Define directory where the results, e.g. json file, will be stored
resultpath = '/media/sf_VBox_Shared/timeseries/PAMAP2_Dataset/results/' 
if not os.path.exists(resultpath):
        os.makedirs(resultpath)

In [14]:
np.concatenate(ys[0:0]+ys[2:]).shape

(12373, 12)

In [16]:
def split_train_val(X_list, y_list, j):
    X_train = np.concatenate(X_list[0:j]+X_list[j+1:])
    X_val = X_list[j]
    y_train = np.concatenate(y_list[0:j]+y_list[j+1:])
    y_val = y_list[j]
    return X_train, y_train, X_val, y_val

In [10]:
import time
t = time.time()
for j in range(len(X_list)):
    X_train, y_train, X_val, y_val = split_train_val(Xs, ys, j)
    histories, val_accuracies, val_losses = find_architecture.train_models_on_samples(X_train, y_train_binary,
                                                                           X_val, y_val_binary,
                                                                           models,nr_epochs=10,
                                                                           subset_size=500,
                                                                           verbose=True,
                                                                           outputfile=resultpath+\
                                                                                  'experiment'+str(j)+'.json')
print(time.time()-t)

Training model 0 DeepConvLSTM
Train on 500 samples, validate on 2007 samples
Epoch 1/10
500/500 [==============================] - 264s - loss: 5.4757 - acc: 0.2360 - val_loss: 1.8126 - val_acc: 0.4056
Epoch 2/10
500/500 [==============================] - 253s - loss: 2.9033 - acc: 0.3260 - val_loss: 1.6804 - val_acc: 0.4056
Epoch 3/10
500/500 [==============================] - 255s - loss: 2.7290 - acc: 0.3340 - val_loss: 1.6941 - val_acc: 0.4175
Epoch 4/10
500/500 [==============================] - 304s - loss: 3.7647 - acc: 0.4260 - val_loss: 1.4799 - val_acc: 0.4150
Epoch 5/10
500/500 [==============================] - 257s - loss: 2.7018 - acc: 0.5000 - val_loss: 2.0925 - val_acc: 0.3607
Epoch 6/10
500/500 [==============================] - 266s - loss: 3.1476 - acc: 0.4420 - val_loss: 2.7559 - val_acc: 0.0583
Epoch 7/10
500/500 [==============================] - 255s - loss: 2.1628 - acc: 0.4980 - val_loss: 1.9769 - val_acc: 0.3009
Epoch 8/10
500/500 [============================

The histories object produced by the previous step contains the history of classifier performance with every iteration of the training process. To ease inspecting this information we developed function plotTrainingProcess, which is demonstrated in the cell below.

Another way of comparing model performance is by putting all the information in a pandas dataframe, which we can store in a csv file.

In [11]:
modelcomparisons = pd.DataFrame({'model':[str(params) for model, params, model_types in models],
                       'train_acc': [history.history['acc'][-1] for history in histories],
                       'train_loss': [history.history['loss'][-1] for history in histories],
                       'val_acc': [history.history['val_acc'][-1] for history in histories],
                       'val_loss': [history.history['val_loss'][-1] for history in histories]
                       })
modelcomparisons.to_csv(resultpath +'modelcomparisons.csv')

modelcomparisons

,model,train_acc,train_loss,val_acc,val_loss
0,"{'filters': array([94]), 'learning_rate': 0.05...",0.706667,11.924376,0.512207,1.655583
1,"{'filters': array([79, 64, 90]), 'learning_rat...",0.923333,6.713034,0.835575,1.017291


It is also possible to vizualize the performance of the various models using our vizualisation tool as explained in the mcfly repository README file: https://github.com/NLeSC/mcfly/blob/master/README.md

Check which model is the best

In [12]:
best_model_index = np.argmax(val_accuracies)
best_model, best_params, best_model_types = models[best_model_index]
print('Model type and parameters of the best model:')
print(best_model_types)
print(best_params)

Model type and parameters of the best model:
CNN
{'regularization_rate': 0.0016693552595679474, 'learning_rate': 0.0013927354361231595, 'filters': array([ 69,  71,  90, 100,  29,  41]), 'fc_hidden_nodes': 1314}


In [13]:
modelname = 'bestmodel_sample'
storage.savemodel(best_model,resultpath,modelname)

('/media/sf_VBox_Shared/timeseries//PAMAP2/PAMAP2_Dataset/results/bestmodel_sample_architecture.json',
 '/media/sf_VBox_Shared/timeseries//PAMAP2/PAMAP2_Dataset/results/bestmodel_sample_weights')

## Train the best model for real

Now that we have identified the best model architecture out of our random pool of models we can continue by training the model on the full training sample. For the purpose of speeding up the example we only train the full model on the first 1000 values. You will need to replace this by 'datasize = X_train.shape[0]' in a real world example.

In [15]:
#We make a copy of the model, to start training from fresh
best_model_copy = modelgen.generate_CNN_model(X_train.shape, num_classes, best_params['filters'], best_params['fc_hidden_nodes'],
                       best_params['learning_rate'], best_params['regularization_rate'])
nr_epochs = 1
datasize = X_train.shape[0] #We're going to train the model on the complete data set
#datasize = 1000 # subsample for the sake of this example
history = best_model_copy.fit(X_train[:datasize,:,:], y_train_binary[:datasize,:],
              nb_epoch=nr_epochs, validation_data=(X_val, y_val_binary))

Train on 12497 samples, validate on 2007 samples
Epoch 1/1
12497/12497 [==============================] - 592s - loss: 2.0438 - acc: 0.9119 - val_loss: 0.2887 - val_acc: 0.9841


In [22]:
# Plot the training process:
find_architecture.plotTrainingProcess(history)

In [20]:
best_model_fullytrained = best_model_copy

In [24]:
score_val = best_model_fullytrained.evaluate(X_val, y_val_binary, verbose=False)
print('Score of best model: ' + str(score_val))

Score of best model: [0.28872849385374272, 0.98405580468360743]


### Saving, loading and comparing reloaded model with orignal model

The modoel can be saved for future use. The savemodel function will save two separate files: a json file for the architecture and a npy (numpy array) file for the weights.

In [7]:
modelname = 'my_bestmodel'

In [18]:
storage.savemodel(best_model_fullytrained,resultpath,modelname)

('/media/sf_VBox_Shared/timeseries//PAMAP2/PAMAP2_Dataset/results/my_bestmodel_architecture.json',
 '/media/sf_VBox_Shared/timeseries//PAMAP2/PAMAP2_Dataset/results/my_bestmodel_weights')

In [8]:
best_model_fullytrained = storage.loadmodel(resultpath,modelname)

In [17]:
best_model_sample = storage.loadmodel(resultpath, 'bestmodel_sample')

FileNotFoundError: [Errno 2] No such file or directory: '/media/sf_VBox_Shared/timeseries/PAMAP2_Dataset/results/bestmodel_sample_architecture.json'

In [9]:
learning_rate = 0.0013927354361231595
best_model_fullytrained.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])

In [10]:
best_model_sample.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])

The model has been reloaded. Let's investigate whether it gives the same probability estimates as the original model in a small subset of the validation data.

## Advanced model inspection

Although beyond the scope of mcfly it may be worth highlighting that the objects 'models', 'best_model_fullytrained' and 'best_model' are Keras objects. This means that you can use Keras functions like .predict and .evaluate on the objects to run advanced analyses. These functions are all documented in the Keras documentation

In [12]:
## Test on Testset
score_test = best_model_fullytrained.evaluate(X_test, y_test_binary, verbose=False)
print('Score of best model: ' + str(score_test))

Score of best model: [1.5646683828375569, 0.52981849611063092]


In [13]:
X_test.shape

(2314, 512, 9)

In [25]:
best_model_sample.evaluate(X_test, y_test_binary, verbose=False)

[1.9208478596103324, 0.46672428694900603]

In [15]:
best_model_fullytrained.evaluate(X_val, y_val_binary, verbose=True)

2007/2007 [==============================] - 40s    


[0.28872849385374272, 0.98405580468360743]

In [11]:
find_architecture.kNN_accuracy(X_train[:500,:,:], y_train_binary[:500,], X_test, y_test_binary, k=1)

0.3595505617977528

In [12]:
find_architecture.kNN_accuracy(X_train[:500,:,:], y_train_binary[:500,], X_val, y_val_binary, k=1)

0.44294967613353264

In [15]:
print(find_architecture.kNN_accuracy(X_train[:1000,:,:], y_train_binary[:1000,], X_test, y_test_binary, k=1))
print(find_architecture.kNN_accuracy(X_train[:1000,:,:], y_train_binary[:1000,], X_val, y_val_binary, k=1))

0.359982713915
0.470852017937
